In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [4]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import torch
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import os

# Define constants
train_root = "/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/train"
test_root = "/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test"
classes = ["benign", "malignant"]

# Create a list to store the paths and labels of all images for training data
train_data = []

# Populate the list with paths and labels for training data
for label, class_name in enumerate(classes):
    folder_path = os.path.join(train_root, class_name)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        train_data.append((image_path, label))

# Create a list to store the paths and labels of all images for test data
test_data = []

# Populate the list with paths and labels for test data
for label, class_name in enumerate(classes):
    folder_path = os.path.join(test_root, class_name)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        test_data.append((image_path, label))

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((700, 460)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((700, 460 )),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 5 # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

import timm

model = timm.create_model("hf_hub:timm/vgg16.tv_in1k", pretrained=True)

# Move model to device
model.to(device)

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [6]:
import torch
import torch.nn as nn
from tqdm import tqdm

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 1/20, Train Loss: 0.6550, Train Accuracy: 69.48%, Val Loss: 0.5159, Val Accuracy: 73.72%


Epoch 2/20, Train Loss: 0.4528, Train Accuracy: 80.75%, Val Loss: 0.4014, Val Accuracy: 86.07%


Epoch 3/20, Train Loss: 0.3985, Train Accuracy: 84.21%, Val Loss: 0.2903, Val Accuracy: 89.95%


Epoch 4/20, Train Loss: 0.3488, Train Accuracy: 85.71%, Val Loss: 0.3282, Val Accuracy: 89.42%


Epoch 5/20, Train Loss: 0.2819, Train Accuracy: 88.38%, Val Loss: 0.8130, Val Accuracy: 56.79%


...

Epoch 16/20, Train Loss: 0.0877, Train Accuracy: 96.63%, Val Loss: 0.6364, Val Accuracy: 83.60%


Epoch 17/20, Train Loss: 0.0873, Train Accuracy: 96.72%, Val Loss: 0.6359, Val Accuracy: 83.77%


Epoch 18/20, Train Loss: 0.0847, Train Accuracy: 96.54%, Val Loss: 0.6368, Val Accuracy: 83.60%


Epoch 19/20, Train Loss: 0.0889, Train Accuracy: 96.63%, Val Loss: 0.6359, Val Accuracy: 83.60%


Epoch 20/20, Train Loss: 0.0905, Train Accuracy: 96.63%, Val Loss: 0.6358, Val Accuracy: 83.60%
Test Accuracy: 83.60%


In [10]:
import torch
from sklearn.metrics import classification_report

# Set model to evaluation mode
model.eval()

# Initialize lists to store the true labels and predicted labels
all_labels = []
all_predictions = []

# Evaluate the model on the test dataset
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Generate the classification report
classification_report_output = classification_report(all_labels, all_predictions)

# Print the classification report
print("Classification Report:")
print(classification_report_output)


Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.74      0.73       171
           1       0.89      0.88      0.88       396

    accuracy                           0.84       567
   macro avg       0.80      0.81      0.81       567
weighted avg       0.84      0.84      0.84       567



In [11]:
# Define a dictionary to store image paths grouped by patient IDs
test_data_by_patient = {}

# Iterate through the test data
for image_path, _ in test_data:
    # Extract the patient ID from the image filename
    patient_id = image_path.split("-")[2]
    # Check if the patient ID already exists in the dictionary
    if patient_id in test_data_by_patient:
        # Append the image path to the list associated with the patient ID
        test_data_by_patient[patient_id].append(image_path)
    else:
        # Create a new list with the image path for the patient ID
        test_data_by_patient[patient_id] = [image_path]

# Print the dictionary containing image paths grouped by patient IDs
for patient_id, images in test_data_by_patient.items():
    print(f"Patient ID: {patient_id}, Number of images: {len(images)}")
    print("Image paths:")
    for image_path in images:
        print(image_path)
    print()


Patient ID: 29315EF, Number of images: 16
Image paths:
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-013.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-014.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-012.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-005.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-009.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-016.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-003.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-006.png
/content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B

In [12]:
# Define a dictionary to store image paths grouped by patient IDs for the training data
train_data_by_patient = {}
# Define a set to store patient IDs for the training data
train_patient_ids = set()

# Iterate through the training data
for image_path, _ in train_data:
    # Extract the patient ID from the image filename
    patient_id = image_path.split("-")[2]
    # Add the patient ID to the set
    train_patient_ids.add(patient_id)
    # Check if the patient ID already exists in the dictionary
    if patient_id in train_data_by_patient:
        # Append the image path to the list associated with the patient ID
        train_data_by_patient[patient_id].append(image_path)
    else:
        # Create a new list with the image path for the patient ID
        train_data_by_patient[patient_id] = [image_path]

# Define a dictionary to store image paths grouped by patient IDs for the test data
test_data_by_patient = {}
# Define a set to store patient IDs for the test data
test_patient_ids = set()

# Iterate through the test data
for image_path, _ in test_data:
    # Extract the patient ID from the image filename
    patient_id = image_path.split("-")[2]
    # Add the patient ID to the set
    test_patient_ids.add(patient_id)
    # Check if the patient ID already exists in the dictionary
    if patient_id in test_data_by_patient:
        # Append the image path to the list associated with the patient ID
        test_data_by_patient[patient_id].append(image_path)
    else:
        # Create a new list with the image path for the patient ID
        test_data_by_patient[patient_id] = [image_path]

# Find common patient IDs between training and test datasets
common_patient_ids = train_patient_ids.intersection(test_patient_ids)

# Find unique patient IDs in the training dataset
unique_train_patient_ids = train_patient_ids.difference(test_patient_ids)

# Find unique patient IDs in the test dataset
unique_test_patient_ids = test_patient_ids.difference(train_patient_ids)

# Print results
print("Number of unique patient IDs in training data:", len(unique_train_patient_ids))
print("Number of unique patient IDs in test data:", len(unique_test_patient_ids))
print("Number of common patient IDs between training and test data:", len(common_patient_ids))

Number of unique patient IDs in training data: 42
Number of unique patient IDs in test data: 25
Number of common patient IDs between training and test data: 0


In [13]:
# Determine the maximum length of the sets to align the printing
max_length = max(len(train_patient_ids), len(test_patient_ids))

# Iterate over the sets simultaneously and print side by side
print("Train Patient IDs\t\tTest Patient IDs")
for train_id, test_id in zip(sorted(train_patient_ids), sorted(test_patient_ids)):
    print(f"{train_id}\t\t\t{test_id}")

# If one set is longer than the other, print the remaining elements
if len(train_patient_ids) > len(test_patient_ids):
    for train_id in sorted(train_patient_ids)[len(test_patient_ids):]:
        print(f"{train_id}\t\t\t")
elif len(test_patient_ids) > len(train_patient_ids):
    for test_id in sorted(test_patient_ids)[len(train_patient_ids):]:
        print(f"\t\t\t{test_id}")

Train Patient IDs		Test Patient IDs
11031			10926
11951			12312
12204			13993
12773			14134
13413			14926
13418DE			15275
14134E			15687B
15570			16184
15570C			16188
15696			16196
15704			16716
15792			17901
16184CD			18842D
16336			190EF
16448			19979
16456			20629
16601			20636
16875			21978AB
17614			22549AB
18650			29315EF
19440			29960AB
19854C			29960CD
19979C			4364
21998AB			5694
21998EF			8168
22549G			
23060AB			
23060CD			
23222AB			
25197			
2523			
2773			
2980			
2985			
3411F			
3909			
4372			
5287			
5695			
6241			
9133			
9461			


In [14]:
# Create a dictionary to store image paths grouped by patient ID
images_by_patient = {}

# Populate the dictionary with image paths grouped by patient ID
for label, class_name in enumerate(classes):
    folder_path = os.path.join(test_root, class_name)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        # Extract patient ID from image file name
        patient_id = image_file.split("-")[2]
        if patient_id not in images_by_patient:
            images_by_patient[patient_id] = []  # Initialize list for patient ID if not already present
        images_by_patient[patient_id].append(image_path)  # Append image path to list for patient ID

# Print images grouped by patient ID
for patient_id, image_paths in images_by_patient.items():
    print("Patient ID:", patient_id)
    for image_path in image_paths:
        print("Image Path:", image_path)
    print()  # Print an empty line for better readability

Patient ID: 29315EF
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-013.png
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-014.png
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-012.png
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-005.png
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-009.png
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-016.png
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-003.png
Image Path: /content/drive/MyDrive/Breast Cancer Project/Akash Split/Mag100/test/benign/SOB_B_PT-14-29315EF-100-006.png
Image Path: /content

In [16]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

# Function to test images of a specific patient ID
def test_patient_images(images):
    # Create a list to store the paths and labels of the images for the patient
    patient_data = []
    for image_path in images:
        label = int(image_path.split("/")[-2] == "malignant")  # Extract label (0 for benign, 1 for malignant)
        patient_data.append((image_path, label))

    # Create a custom dataset for the patient's images
    patient_dataset = CustomDataset(patient_data, transform=test_val_transform)

    # DataLoader for the patient's images
    patient_loader = DataLoader(patient_dataset, batch_size=batch_size, shuffle=False)

    # Test the model on patient's images
    model.eval()
    all_predicted = []
    all_labels = []
    with torch.no_grad():
        for images, labels in patient_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())


    # Return predictions and labels for the patient's images
    return all_labels, all_predicted

# List of patient IDs
patient_ids = [
    "10926", "12312", "13993", "14134", "14926", "15275", "15687B", "16184", "16188",
    "16196", "16716", "17901", "18842D", "190EF", "19979", "20629", "20636", "21978AB",
    "22549AB", "29315EF", "29960AB", "29960CD", "4364", "5694", "8168"
]

# Initialize lists to store all predictions and labels
all_predictions = []
all_labels = []

# Iterate through each patient ID
for patient_id in patient_ids:
    # Test images for the patient ID
    patient_images = images_by_patient.get(patient_id, [])
    labels, predictions = test_patient_images(patient_images)
    all_labels.extend(labels)
    all_predictions.extend(predictions)

# Calculate the average classification report
average_classification_report = classification_report(all_labels, all_predictions, digits=3)

# Print the average classification report
print("Average Classification Report Across All Patient IDs:")
print(average_classification_report)

Average Classification Report Across All Patient IDs:
              precision    recall  f1-score   support

           0      0.722     0.743     0.732       171
           1      0.887     0.876     0.882       396

    accuracy                          0.836       567
   macro avg      0.805     0.809     0.807       567
weighted avg      0.837     0.836     0.837       567



In [18]:
from sklearn.metrics import accuracy_score

# Iterate through each patient ID
for patient_id in patient_ids:
    # Test images for the patient ID
    patient_images = images_by_patient.get(patient_id, [])
    labels, predictions = test_patient_images(patient_images)

    # Calculate accuracy for the patient ID
    accuracy = accuracy_score(labels, predictions)

    # Print accuracy for the patient ID
    print(f"Accuracy for Patient ID {patient_id}: {accuracy:.2f}")

Accuracy for Patient ID 10926: 0.40
Accuracy for Patient ID 12312: 0.46
Accuracy for Patient ID 13993: 1.00
Accuracy for Patient ID 14134: 0.90
Accuracy for Patient ID 14926: 0.89
Accuracy for Patient ID 15275: 1.00
Accuracy for Patient ID 15687B: 1.00
Accuracy for Patient ID 16184: 0.53
Accuracy for Patient ID 16188: 1.00
Accuracy for Patient ID 16196: 1.00
Accuracy for Patient ID 16716: 1.00
Accuracy for Patient ID 17901: 1.00
Accuracy for Patient ID 18842D: 1.00
Accuracy for Patient ID 190EF: 1.00
Accuracy for Patient ID 19979: 1.00
Accuracy for Patient ID 20629: 0.88
Accuracy for Patient ID 20636: 0.65
Accuracy for Patient ID 21978AB: 1.00
Accuracy for Patient ID 22549AB: 0.77
Accuracy for Patient ID 29315EF: 1.00
Accuracy for Patient ID 29960AB: 0.00
Accuracy for Patient ID 29960CD: 0.77
Accuracy for Patient ID 4364: 0.80
Accuracy for Patient ID 5694: 1.00
Accuracy for Patient ID 8168: 1.00


In [19]:
from sklearn.metrics import accuracy_score

# Initialize variables to accumulate total accuracy and total patient IDs
total_accuracy = 0
total_patient_ids = 0

# Iterate through each patient ID
for patient_id in patient_ids:
    # Test images for the patient ID
    patient_images = images_by_patient.get(patient_id, [])
    labels, predictions = test_patient_images(patient_images)

    # Calculate accuracy for the patient ID
    accuracy = accuracy_score(labels, predictions)

    # Accumulate total accuracy and total patient IDs
    total_accuracy += accuracy
    total_patient_ids += 1

# Calculate the average accuracy
average_accuracy = total_accuracy / total_patient_ids

# Print the average accuracy
print(f"Average Accuracy Across All Patient IDs: {average_accuracy:.2f}")

Average Accuracy Across All Patient IDs: 0.84
